In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib

%matplotlib inline
sns.set(style = 'white', font_scale = 0.9)


import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import scikitplot as skplt

In [2]:
def classifier_mod(X_train, y_train, X_test, y_test, sampName):
    
    ## print the shape of imported dataset
    print('X_train shape: ', X_train.shape)
    print('y_train shape: ', y_train.shape)
    print('X_test shape: ', X_test.shape)
    print('y_test shape: ', y_test.shape)
    
    ## grid search for the optimal hyperparameters
    model = XGBClassifier(random_state = 123, objective = 'binary:logistic', nthread = 4, seed = 123) #scale_pos_weight
    
    param_grid = {'n_estimators': [100,200,300,400,500], 
                  'learning_rate': [0.01, 0.1, 0.5],
                 'max_depth': [15, 20, 25, 30], 
                  'min_child_weight': [1,2,3], 
                  'gamma': [0.1,0.2],
                 'colsample_bytree': [0.6,0.7], 
                  'subsample': [0.6, 0.7], 
                  'reg_alpha': [1e-5,1e-2,0.1,1,100]}
    
    grid = GridSearchCV(estimator = model, 
                        param_grid = param_grid, 
                        scoring = 'roc_auc', 
                        n_jobs = -1, 
                        verbose = 1)
    
    grid.fit(X_train, y_train)
    
    print(grid.best_score_)
    print(grid.best_params_)
    
    ## fit the model with the optimal hyperparameters
    classifier = XGBClassifier(random_state = 123,
                               n_estimators = grid.best_params_['n_estimators'],
                               learning_rate = grid.best_params_['learning_rate'],
                               max_depth = grid.best_params_['max_depth'],
                               min_child_weight = grid.best_params_['min_child_weight'],
                               gamma = grid.best_params_['gamma'],
                               colsample_bytree = grid.best_params_['colsample_bytree'],
                               subsample = grid.best_params_['subsample'],
                               reg_alpha = grid.best_params_['reg_alpha'])
    
    classifier.fit(X_train, y_train)
    
    ## pickle the fitted model
    joblib.dump(classifier, 'xgboost_'+sampName+'.pkl')
    
    ## pickle the predicted responses
    y_pred = classifier.predict(X_test)
    y_pred_prob = classifier.predict_proba(X_test)
    
    joblib.dump(y_pred, 'y_pred_'+sampName+'.pkl')
    joblib.dump(y_pred_prob, 'y_pred_prob_'+sampName+'.pkl')
    
    # printout metrics for model performance
    print('Accuracy score: ', accuracy_score(y_test, y_pred))
    
    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
    skplt.metrics.plot_confusion_matrix(y_test, y_pred)
    plt.savefig('confusion_matrix_xgboost.png')
    
    skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize = True)
    plt.savefig('confusion_matrix_xgboost_norm.png')
    
    
    print('Classification report:\n', classification_report(y_test, y_pred))
    
    print('ROC_AUC_score: ', roc_auc_score(y_test, y_pred))
    skplt.metrics.plot_roc(y_test, y_pred_prob)
    plt.savefig('roc_xgboost.png')
    
    skplt.metrics.plot_precision_recall_curve(y_test, y_pred_prob)
    plt.savefig('pr_xgboost.png')
    
    # printout feature importances
    feature_import = pd.DataFrame(data = classifier.feature_importances_, index = X_test.columns.values, columns = ['values'])
    feature_import.sort_values(['values'], ascending = False, inplace = True)
    feature_import.reset_index(level = 0, inplace = True)
    print(feature_import)
    
    # feature importances plot
    plt.figure(figsize = (35,20))
    sns.set(style = 'white', font_scale = 2)
    
    ax = sns.barplot(x = 'values', y = 'index', data = feature_import, palette = 'deep')
    ax.set_title('Variables Imporatnce Plot of XGBoost')
    plt.savefig('importance_xgboost.png')
    
    return classifier


In [3]:
import os

os.chdir('C:/Users/LUY1/Desktop/customer churn data/data')
print(os.getcwd())

C:\Users\LUY1\Desktop\customer churn data\data


In [4]:
X_train = pd.read_pickle('X_train.pkl')
X_test = pd.read_pickle('X_test.pkl')

y_train = pd.read_pickle('y_train.pkl')
y_test = pd.read_pickle('y_test.pkl')

In [5]:
os.chdir('C:/Users/LUY1/Desktop/customer churn data/models')
print(os.getcwd())

C:\Users\LUY1\Desktop\customer churn data\models


In [ ]:
classifier_mod(X_train, y_train, X_test, y_test, 'nosamp')

X_train shape:  (419420, 19)
y_train shape:  (419420,)
X_test shape:  (104855, 19)
y_test shape:  (104855,)
Fitting 3 folds for each of 7200 candidates, totalling 21600 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 42.1min
